# Multimodal Integration

Combine documents and audio in a unified pipeline.

## Setup

In [ ]:
from pathlib import Path
from vertector_data_ingestion import (
    UniversalConverter,
    LocalMpsConfig,
    HybridChunker,
    ChromaAdapter,
    create_audio_transcriber,
    AudioConfig,
    WhisperModelSize,
    setup_logging,
)

setup_logging(log_level="INFO")

## Multimodal Pipeline

In [ ]:
class MultimodalPipeline:
    def __init__(self):
        self.converter = UniversalConverter(LocalMpsConfig())
        self.chunker = HybridChunker()
        self.audio_transcriber = create_audio_transcriber(
            AudioConfig(model_size=WhisperModelSize.BASE)
        )
        self.vector_store = ChromaAdapter(collection_name="multimodal")
    
    def process_document(self, path: Path):
        print(f"Processing: {path.name}")
        doc = self.converter.convert(path)
        chunks = self.chunker.chunk_document(doc.document)
        
        for chunk in chunks.chunks:
            chunk.metadata["modality"] = "document"
            chunk.metadata["source"] = path.name
        
        self.vector_store.add_chunks(chunks.chunks)
        print(f"  Added {len(chunks.chunks)} chunks")
        return len(chunks.chunks)
    
    def process_audio(self, path: Path):
        print(f"Processing: {path.name}")
        result = self.audio_transcriber.transcribe(path)
        
        from vertector_data_ingestion.models.chunk import DocumentChunk
        chunks = []
        for i, segment in enumerate(result.segments):
            chunk = DocumentChunk(
                chunk_id=f"{path.stem}_{i}",
                text=segment.text,
                metadata={
                    "modality": "audio",
                    "source": path.name,
                    "start_time": segment.start,
                    "end_time": segment.end,
                }
            )
            chunks.append(chunk)
        
        self.vector_store.add_chunks(chunks)
        print(f"  Added {len(chunks)} chunks")
        return len(chunks)
    
    def search(self, query: str, top_k: int = 5):
        return self.vector_store.search(query, top_k=top_k)

pipeline = MultimodalPipeline()
print("✓ Pipeline ready")

## Process Documents and Audio

In [ ]:
# Process documents
doc_path = Path("document.pdf")
if doc_path.exists():
    pipeline.process_document(doc_path)

# Process audio
audio_path = Path("audio.wav")
if audio_path.exists():
    pipeline.process_audio(audio_path)

## Cross-Modal Search

In [ ]:
results = pipeline.search("machine learning", top_k=3)

for i, result in enumerate(results, 1):
    modality = result['metadata'].get('modality', 'unknown')
    source = result['metadata'].get('source', 'unknown')
    
    print(f"\nResult {i} [{modality.upper()}]:")
    print(f"  Source: {source}")
    print(f"  Text: {result['text'][:100]}...")

## Summary

Demonstrated:
- Unified multimodal pipeline
- Document and audio processing
- Cross-modal search

See documentation for more examples.